In [1]:
# dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
from pprint import pprint
import requests
import PyPDF2
import os
import io

In [2]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
# start browser
executable_path = {'executable_path':'chromedriver.exe'}
browser = Browser('chrome',**executable_path,headless=True)

In [4]:
# check for working links
arr = ['2010','2011','2012','2013','2014','2015','2016']
url = 'https://research.collegeboard.org/programs/sat/data/archived/cb-seniors-'
for i in arr:
    print(url+i)

https://research.collegeboard.org/programs/sat/data/archived/cb-seniors-2010
https://research.collegeboard.org/programs/sat/data/archived/cb-seniors-2011
https://research.collegeboard.org/programs/sat/data/archived/cb-seniors-2012
https://research.collegeboard.org/programs/sat/data/archived/cb-seniors-2013
https://research.collegeboard.org/programs/sat/data/archived/cb-seniors-2014
https://research.collegeboard.org/programs/sat/data/archived/cb-seniors-2015
https://research.collegeboard.org/programs/sat/data/archived/cb-seniors-2016


In [5]:
# DEPRECATED, skip to next block
# grab scores from overall mean scores (page 5)
arr = ['2010','2011']

url = 'https://research.collegeboard.org/programs/sat/data/archived/cb-seniors-'
for i in arr:
    browser.visit(url+i)
    html = browser.html
    soup = bs(html,'html.parser')
    
    table = soup.find('table',class_='table')
    states = table.find_all('a')
    
    print(f'for year {i}')
    print('----------------------')

    for state in states:
        state_name = state.text
        state_url = state['href']
        response = requests.get(state_url)
        
        with io.BytesIO(response.content) as open_pdf_file:
            pdf_reader = PyPDF2.PdfFileReader(open_pdf_file)
            score_page = pdf_reader.getPage(4)
            text = score_page.extractText().split('\n')
            
            first_loc = [i for i, s in enumerate(text) if 'SDMeanSDMeanTotal' in s][0]
            number_str = text[first_loc][-18:]
            scores = [number_str[i:i+3] for i in range(0, len(number_str), 3)]
            
            # uncomment to view output
            # print(scores)

for year 2010
----------------------
for year 2011
----------------------


In [6]:
# grab scores from 75th percentile for state (page 6)
arr = ['2010','2011']
url = 'https://research.collegeboard.org/programs/sat/data/archived/cb-seniors-'

year_holder = {}
for i in arr:
    browser.visit(url+i)
    html = browser.html
    soup = bs(html,'html.parser')
    
    table = soup.find('table',class_='table')
    states = table.find_all('a')
    
    print(f'for year {i}')
    print('----------------------')
    
    state_holder = {}
    for state in states:
        state_name = state.text
        state_url = state['href']
        response = requests.get(state_url)
        
        with io.BytesIO(response.content) as open_pdf_file:
            pdf_reader = PyPDF2.PdfFileReader(open_pdf_file)
            score_page = pdf_reader.getPage(5)
            text = score_page.extractText().split('\n')
            
            first_loc = [i for i, s in enumerate(text) if 'Critical Reading' in s][1]+1
            number_str = text[first_loc]
            scores = [number_str[i:i+3] for i in range(0, len(number_str), 3)]

            # uncomment to view output
            # print(f'{scores} ==> {state_name}')
            
            state_holder[state_name] = scores
            
    year_holder[i] = state_holder
    
print('finished.')

for year 2010
----------------------
for year 2011
----------------------
finished.


In [7]:
pprint(year_holder)

{'2010': {'Alabama': ['640', '630', '620'],
          'Alaska': ['590', '590', '560'],
          'Arizona': ['580', '590', '560'],
          'Arkansas': ['650', '640', '620'],
          'California': ['580', '600', '570'],
          'Colorado': ['640', '650', '620'],
          'Connecticut': ['580', '600', '590'],
          'Delaware': ['560', '570', '550'],
          'District of Columbia': ['580', '580', '580'],
          'Florida': ['560', '570', '550'],
          'Georgia': ['560', '560', '540'],
          'Hawaii': ['550', '580', '540'],
          'Idaho': ['610', '610', '580'],
          'Illinois': ['670', '690', '660'],
          'Indiana': ['560', '570', '540'],
          'Iowa': ['690', '690', '660'],
          'Kansas': ['670', '670', '640'],
          'Kentucky': ['650', '650', '630'],
          'Louisiana': ['640', '640', '630'],
          'Maine': ['540', '540', '530'],
          'Maryland': ['580', '590', '570'],
          'Massachusetts': ['590', '610', '590'],
        

In [8]:
# remember to close browser
browser.quit()